# TensorFlow/Kerasによる分類(予測)


In [ ]:
# ライブラリのインポート
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import numpy as np

from keras.preprocessing.image import load_img
from keras.models import load_model

import cv2, matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# 各種パラメータ
img_rows, img_cols = 64, 64
batch_size = 128
num_classes = 3
arduino_led_enable = False 

In [ ]:
# 学習したモデルの読み込み
model = load_model("data/imas_yukiho_miki_yayoi.h5")

# 顔検出器の初期化
cascade = cv2.CascadeClassifier("data/lbpcascade_animeface.xml") # アニメ顔検出器の読み込み

# Arduionoとの通信用シリアルポートの初期化
ser = None
if arduino_led_enable:
    import serial
    ser = serial.Serial(port='/dev/cu.usbmodem1431', baudrate=9600) # portは実際の環境に合わせること


In [ ]:
# 予測用関数の定義
def image_predict(image_path):
    img = cv2.imread(image_path)
    gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) # イメージをグレースケールに変換
    cv2.equalizeHist(gray_img, gray_img) # ヒストグラムを平坦化
    facerect = cascade.detectMultiScale(gray_img, scaleFactor=1.1, minNeighbors=1, minSize=(1, 1)) # 顔認識

    # 認識したものの中で、最大サイズの矩形を選択
    maxsize = 0
    maximagerect = None
    if len(facerect) > 0:
        for rect in facerect:
            if maxsize < rect[2] * rect[3]:
                maxsize = rect[2] * rect[3]
                maximagerect = rect
    
    # 認識した顔が存在する場合、予測を実施する
    if maximagerect is not None:
        img_x = maximagerect[0]
        img_y = maximagerect[1]
        img_w = maximagerect[2]
        img_h = maximagerect[3]
        
        small_img = cv2.resize(img[img_y:img_y+img_h, img_x:img_x+img_w], (img_cols, img_rows))

        small_img_rgb = cv2.cvtColor(small_img, cv2.COLOR_BGR2RGB)
        im_reading = np.array(small_img_rgb)
        pred_data_set = np.empty((0,img_cols,img_rows,3), dtype=np.uint8)
        pred_data_set = np.append(pred_data_set, [im_reading], axis=0)
        
        y_list = model.predict(pred_data_set, batch_size=batch_size, verbose=0)
        for i in range(len(y_list)):
            
            print("雪歩: %1.20f" % y_list[i][0])
            print("美希: %1.20f" % y_list[i][1])
            print("やよい: %1.20f" % y_list[i][2])
            
            # Arduionoに信号を送る
            if y_list[i][0]>0.8:
                output_bytes = b'1'
            else:
                output_bytes = b'0'
            if ser != None and ser.isOpen():
                ser.write(output_bytes)
                
            preview_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            plt.imshow(preview_img)    


In [ ]:
image_predict('data/sample/yukiho1.jpg') 

In [ ]:
image_predict('data/sample/yukiho2.jpg') 

In [ ]:
image_predict('data/sample/yukiho3.jpg') 

In [ ]:
image_predict('data/sample/yukiho4.jpg') 

In [ ]:
image_predict('data/sample/yukiho5.jpg') 

In [ ]:
image_predict('data/sample/yayoi1.jpg') 

In [ ]:
image_predict('data/sample/yayoi2.jpg') 

In [ ]:
image_predict('data/sample/miki1.jpg') 